In [1]:
import os 
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm 
import math
from deep_translator import GoogleTranslator
from transformers import AutoTokenizer, AutoModelWithLMHead , AutoModelForSeq2SeqLM

In [2]:
tokenizer=AutoTokenizer.from_pretrained('T5-base')
model=AutoModelWithLMHead.from_pretrained('T5-base', return_dict=True)
def summarize_transformer_english(sentences):
    inputs=tokenizer.encode("sumarize: " +"".join(sentences),return_tensors='pt', max_length=512, truncation=True)
    output = model.generate(inputs, min_length=80, max_length=100)
    summary=tokenizer.decode(output[0])
    summary=summary.replace('<pad>','')
    summary=summary.replace('</s>','')
    summary=summary.replace('<unk>','<')
    return summary

tokenizer_hindi = AutoTokenizer.from_pretrained("rahular/varta-t5")
model_hindi = AutoModelForSeq2SeqLM.from_pretrained("rahular/varta-t5")

def summarize_transformer_hindi(sentences):
    inputs=tokenizer_hindi.encode("summarize: " +"".join(sentences),return_tensors='pt', max_length=512, truncation=True)
    output = model_hindi.generate(inputs, min_length=80, max_length=100)
    summary=tokenizer_hindi.decode(output[0])
    summary=summary.replace('<pad>','')
    summary=summary.replace('</s>','')
    summary=summary.replace('<unk>','<')
    summary= summary.strip(' ')
    return summary

/Users/manoj/miniconda3/envs/hldc/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [3]:
def split_long_sentence(sentence, max_length=3000):
    parts = []
    for i in range(0, len(sentence), max_length):
        part = sentence[i:i + max_length]
        parts.append(part)

    return parts

def convertToHindi(sentences):
    hindi = []
    for sentence_array in sentences:
        sentence = ''
        hindiText=''
        for i in range(len(sentence_array)):
            sentence = sentence + sentence_array[i]
        if len(sentence)>2000:
            small_sentences = split_long_sentence(sentence, max_length=2000)
            hindiPart = ''
            for i in range(len(small_sentences)):
                current_translate = GoogleTranslator(source='en', target='hi').translate(small_sentences[i])
                hindiPart = hindiPart + current_translate
        else:
            hindiPart = GoogleTranslator(source='en', target='hi').translate(sentence)
        hindiText=hindiText + hindiPart
        hindi.append(hindiText)
    return hindi

In [4]:
input_path="./data/summary/"
output_path="./data/summary_results/"
if os.path.exists(output_path) == False:
    os.mkdir(output_path)

data = pd.read_csv(f'{input_path}data.csv')
sentences = data['sentences'].apply(eval)
sentences_english = data['sentences_english'].apply(eval)

summary=[]
summary_english=[]
for i in tqdm(range(len(sentences))):
    summary.append(summarize_transformer_hindi( sentences[i]))
    summary_english.append(convertToHindi((summarize_transformer_english(  sentences_english[0])).split(".")))

data['summary'] = summary
data['summary_english'] = summary_english

data.to_csv(f'{output_path}transformer.csv', index=False)

  0%|          | 0/1000 [00:00<?, ?it/s]